In [11]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [12]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [13]:
# extract text from given .pdf file
def pdf2text(pdf_path, page_start, page_end):
    # creating a pdf reader object
    reader = PyPDF2.PdfReader(pdf_path)
    
    # printing number of pages in pdf file
    page_max = len(reader.pages)
    
    # getting a specific page from the pdf file
    text = ""
    
    for i in range(page_start, page_end + 1):
        # print(i)
        page = reader.pages[i]
        text = text + "".join(page.extract_text().splitlines())

    return text
# --------------------start of test code--------------------
# pdf_folder_path = fpath.litera_pdf_folder
# pdf_file_name = "1.pdf"
# pdf_path = os.path.join(pdf_folder_path, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# text = pdf2text(pdf_path, start_page, end_page)

# with open(text_path, "w") as f:
#     f.write(text)

# f.close()
# ---------------------end of test code---------------------

In [14]:
# download pdf to specified folder given pdf_url and file name
def download_pdf(pdf_url, pdf_folder_path, file_name):    
    # get the pdf content
    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
    "download.default_directory": pdf_folder_path, #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
    })
    driver = webdriver.Chrome(options=options)
    response  = driver.get(pdf_url)
    time.sleep(5)
    # response = requests.get(pdf_url, stream=True, headers=plib.headers)
    
    # download the .pdf file to the pdf_file_path folder
    # write content in pdf file
    # pdf_path = os.path.join(pdf_folder_path, file_name + '.pdf')
    
    # if response.status_code == 200:
    #     with open(pdf_path, 'wb') as pdf_object:
    #         pdf_object.write(response.content)
    #         return True
    # else:
    #     print(f'Failed downloading PDF:' + 'pdf_url')
    #     print(f'HTTP response status code: {response.status_code}')
    #     return False
# --------------------start of test code--------------------
# pdf_url = 'https://www.sciencedirect.com/science/article/pii/S0896627320300052/pdfft?md5=3f0648c6385e6fae3a5a73b053903014&pid=1-s2.0-S0896627320300052-main.pdf'
# pdf_folder_path = fpath.litera_pdf_folder
# file_name = '4'
# download_pdf(pdf_url, pdf_folder_path, file_name)
# ---------------------end of test code---------------------